In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

In [2]:
#read in data
data_path = "/home/kishen/documents/python_projects/disaster_tweets/data/"
df = pd.read_csv(data_path + "clean_tweets.csv")

Preprocessing

In [3]:
os.chdir("/home/kishen/documents/python_projects/disaster_tweets/")
from src.processing import tokenizer

from sklearn.model_selection import train_test_split

#Split data
X, y = df['text'], df['target'].values



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .2, shuffle=True, random_state=24)

#Sanity Check
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8978,) (8978,)
(2245,) (2245,)


In [4]:
X_train[:3]

/tmp/ipykernel_15388/1529872051.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  X_train[:3]


9160     1/ THANK YOU IN ADVANCE FOR YOUR PATIENCE whil...
2590     BBC getting their election collision payoff in...
10363    No. That is not the thing she likes.once slept...
Name: text, dtype: object

In [5]:
from src.datasets import Dataset, Vocabulary, MyCollate

train_loader = Dataset(X_train, y_train, freq_threshold=2)


In [7]:
from torch.utils.data import Dataset, DataLoader

# Parameters
params = {'batch_size': 16,
          'shuffle': True,
          'num_workers': 8,
          'collate_fn':MyCollate(train_loader.vocab.stoi["<PAD>"])}

In [9]:
loader = DataLoader(dataset=train_loader, 
                    **params)

In [10]:
epochs = 5

for epoch in range(epochs):
    for batch, label in train_loader:
        print(batch)
        print(label)
        break



tensor([   1,  842,  694,    6,  866,  867, 4553,  105, 8068,   14, 1778, 1554,
          17,  184, 1554,   17,  843,  105,  844, 1259,    2])
tensor(0)
tensor([   1,  842,  694,    6,  866,  867, 4553,  105, 8068,   14, 1778, 1554,
          17,  184, 1554,   17,  843,  105,  844, 1259,    2])
tensor(0)
tensor([   1,  842,  694,    6,  866,  867, 4553,  105, 8068,   14, 1778, 1554,
          17,  184, 1554,   17,  843,  105,  844, 1259,    2])
tensor(0)
tensor([   1,  842,  694,    6,  866,  867, 4553,  105, 8068,   14, 1778, 1554,
          17,  184, 1554,   17,  843,  105,  844, 1259,    2])
tensor(0)
tensor([   1,  842,  694,    6,  866,  867, 4553,  105, 8068,   14, 1778, 1554,
          17,  184, 1554,   17,  843,  105,  844, 1259,    2])
tensor(0)
